In [4]:
import re, os
#import mosestokenizer
from string import punctuation
import pandas as pd

In [12]:
def ordered_string_join(x, j=''):
    s = sorted(x, key=lambda x: x[0])
    a,b = list(zip(*s))
    return a, j.join(b)

In [3]:
# s\w\d+ma2: data in fixation order
# WNUM in eyetracking DF maps to word index in text DF
dundeeDir = 'eye-tracking'
fileList = [os.path.join(dundeeDir, f) for f in os.listdir(dundeeDir) if re.match(r's\w\d+ma2p*\.dat', f)]
cols = ['WorkerId', 'text_id', 'WORD','TEXT','LINE','OLEN','WLEN','XPOS','WNUM','FDUR','OBLP','WDLP','FXNO','TXFR']
dundee = pd.DataFrame(columns = cols)
for file in fileList:
    temp = pd.read_csv(file, sep='\s+', encoding='Windows-1252')
    match = re.search(r'(s\w)(\d+)ma2p*\.dat', file.split('/')[-1])
    subjId = match.group(1)
    text = int(match.group(2))
    temp.insert(loc=0, column='text_id', value=text)
    temp.insert(loc=0, column='WorkerId', value=subjId)
    dundee = dundee.append(temp)
dundee.rename(columns = {'FDUR':'time', 'WORD':'word', 'WNUM': 'Word_Number'}, inplace = True)
dundee['time'] = dundee.time.astype('int64')
dundee = dundee.reset_index().drop(columns=['index','OLEN','XPOS','OBLP','WDLP','FXNO','TXFR'])

In [9]:
dundee.head(30)

,WorkerId,text_id,word,TEXT,LINE,WLEN,Word_Number,time
0,sg,13,The,1,1,3,1,148
1,sg,13,renaissance,1,1,11,2,155
2,sg,13,renaissance,1,1,11,2,93
3,sg,13,in,1,1,2,3,61
4,sg,13,children's,1,1,10,4,201
5,sg,13,children's,1,1,10,4,223
6,sg,13,children's,1,1,10,4,205
7,sg,13,literature,1,1,10,5,53
8,sg,13,literature,1,1,10,5,164
9,sg,13,is,-99,0,0,6,0


In [29]:

dundeeDir = 'texts'
textList = [os.path.join(dundeeDir, f) for f in os.listdir(dundeeDir) if re.match(r'tx\d+wrdp\.dat', f)]
cols = ['word', 'text_id', 'screen_nr', 'line_nr', 'pos_on_line', 'serial_nr', 'initial_letter_position', 'word_len_punct', 'word_len', 'punc_code', 'n_chars_before','n_chars_after', 'Word_Number', 'local_word_freq']
dundeeTexts = pd.DataFrame(columns = cols)
for text in textList:
    temp = pd.read_csv(text, sep='\s+', names=cols, encoding='Windows-1252')
    dundeeTexts = dundeeTexts.append(temp)
dundee['word'] = dundee.apply(lambda x: re.sub(r"\s+", ' ', MOSESNORMALIZER(x['word'].strip().replace('""','"').replace('\n',' ')).lower()), axis=1)

dundeeTexts.head()



,word,text_id,screen_nr,line_nr,pos_on_line,serial_nr,initial_letter_position,word_len_punct,word_len,punc_code,n_chars_before,n_chars_after,Word_Number,local_word_freq
0,The,7,1,1,1,1,0,3,3,0,0,0,1,3511
1,case,7,1,1,2,2,4,4,4,0,0,0,2,26
2,of,7,1,1,3,3,9,2,2,0,0,0,3,1757
3,Ms,7,1,1,4,4,12,2,2,0,0,0,4,9
4,Susan,7,1,1,5,5,15,5,5,0,0,0,5,3


In [31]:
dundeeTexts.to_csv("./dundee_text.csv")

In [25]:
inds, paragraphs = zip(*dundeeTexts[['text_id','Word_Number','word']].drop_duplicates().dropna().groupby(by = ['text_id']).apply(lambda x: ordered_string_join(zip(x['Word_Number'], x['word']), ' ')))
#dundee_stats = corpus_stats(list(enumerate(paragraphs,1)), models=DUNDEE_MODELS)

In [26]:
paragraphs[0]

"Are tourists enticed by these attractions threatening their very existence? The two young sea-lions took not the slightest interest in our arrival. They were playing on the jetty, rolling over and tumbling into the water together, entirely ignoring the human beings edging awkwardly round them. Our party then had to step gingerly over hundreds of marine iguanas basking on the rocks, strange creatures with richly coloured velvet coats, looking like something between a giant lizard and a miniature dinosaur, and found nowhere else on earth except here in the Galapagos. A little further on were the blue-footed boobies, birds with brilliant china-blue feet, again unique. And then there were the flightless cormorants, birds with vestigial wings who, over the generations, have lost the ability to fly because the nutrients at the shoreline are so rich they have no need to leave. The island of Fernandina lies at the far western edge of the Galapagos archipelago, dominated by a volcano that last

In [7]:
# s\w\d+ma1: data in fixation order
# WNUM in eyetracking DF maps to word index in text DF
dundeeDir = 'eye-tracking'
fileList = [os.path.join(dundeeDir, f) for f in os.listdir(dundeeDir) if re.match(r's\w\d+ma1p*\.dat', f)]
cols = ['WorkerId', 'text_id', 'WORD','TEXT','LINE','OLEN','WLEN','XPOS','WNUM','FDUR','OBLP','WDLP','FXNO','TXFR']
dundee = pd.DataFrame(columns = cols)
for file in fileList:
    temp = pd.read_csv(file, sep='\s+', encoding='Windows-1252')
    match = re.search(r'(s\w)(\d+)ma1p*\.dat', file.split('/')[-1])
    subjId = match.group(1)
    text = int(match.group(2))
    temp.insert(loc=0, column='text_id', value=text)
    temp.insert(loc=0, column='WorkerId', value=subjId)
    dundee = dundee.append(temp)
dundee.rename(columns = {'FDUR':'time', 'WORD':'word', 'WNUM': 'Word_Number'}, inplace = True)
dundee['time'] = dundee.time.astype('int64')
dundee = dundee.reset_index().drop(columns=['index','OLEN','XPOS','OBLP','WDLP','FXNO','TXFR'])

In [8]:
dundee.head(20)

,WorkerId,text_id,word,TEXT,LINE,WLEN,Word_Number,time,LAUN
0,sj,20,After,1,1,5,1,201,0.0
1,sj,20,After,1,1,5,1,260,0.0
2,sj,20,England's,1,1,9,2,181,-8.0
3,sj,20,overwhelming,1,1,12,3,222,-9.0
4,sj,20,defeat,1,1,6,4,228,-12.0
5,sj,20,the,1,1,3,6,198,-9.0
6,sj,20,admired,1,1,7,7,185,-6.0
7,sj,20,Ireland,1,1,7,8,198,-6.0
8,sj,20,admired,1,1,7,7,299,8.0
9,sj,20,how,1,1,3,10,178,-16.0


In [9]:
dundee.to_csv("./dundee_eyetracking.csv")